<a href="https://colab.research.google.com/github/rootUJ99/hf_transformer_diffusers/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Text Classification notebook

In [1]:
%pip install transformers==4.28 datasets evaluate

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
dataset["train"][101]

{'text': 'Assuming this won\'t end up a straight-to-video release, I would have to say void this title at all costs. Unless you\'re bored of good, well-executed movies, that is. I saw this last night at AFI Dallas, and I left with 20 minutes remaining, simply because I didn\'t care anymore (about the plot, not about insulting the director...that is awkward). When you can spot a goof only 5 minutes into the movie (a shot out, shattered window before any shots are fired...and then the window breaks with the first shot), things are going to bad. Let\'s just say this is only an indicator of things to come...unfortunately.<br /><br />I\'ll spare you all the details, but this is sub-par in every manner, even the half-assed acting by Michael Madsen is disappointing when you\'re expecting half-assed acting from him. And the rape scene...Christ! "Shut up and take it" should never be used in a rape scene. EVER.<br /><br />3/10',
 'label': 0}

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:
def callback_fn(samples):
    return tokenizer(samples['text'], truncation=True)
tokenized_dataset = dataset.map(callback_fn, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

In [10]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
id2label = {0:"NEGATIVE", 1:"POSITIVE"}
label2id = {"NEGATIVE":0, "POSITIVE":1}

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [13]:
training_args = TrainingArguments(
    output_dir="first_classification_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/rootuj/first_classification_model into local empty directory.


In [15]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.232400,0.193633,0.927040
2,0.152600,0.237919,0.929720


TrainOutput(global_step=3126, training_loss=0.20856353615768736, metrics={'train_runtime': 3373.3655, 'train_samples_per_second': 14.822, 'train_steps_per_second': 0.927, 'total_flos': 6561288258498624.0, 'train_loss': 0.20856353615768736, 'epoch': 2.0})

inference

In [16]:
trainer.push_to_hub()

Upload file runs/Aug11_20-20-49_ffa096799b71/events.out.tfevents.1691785351.ffa096799b71.2192.0:   0%|        …

To https://huggingface.co/rootuj/first_classification_model
   155eb2f..d40411e  main -> main

   155eb2f..d40411e  main -> main

To https://huggingface.co/rootuj/first_classification_model
   d40411e..f5495d7  main -> main

   d40411e..f5495d7  main -> main



'https://huggingface.co/rootuj/first_classification_model/commit/d40411e723635da00bcbbfc42bd81314c23b07bf'

In [17]:
from transformers import pipeline

classifier = pipeline("text-classification", model="rootuj/first_classification_model")

In [28]:
text = dataset['test'][100]['text']
label = dataset['test'][100]['label']

label, text

(0,
 '"Cover Girl" is a lacklustre WWII musical with absolutely nothing memorable about it, save for its signature song, "Long Ago and Far Away." This film came out before Gene Kelly really hit his artistic stride, and while there are evidences of his burgeoning talent here, mostly he plays sidekick to Rita Hayworth. And there\'s the problem. Rita Hayworth is gorgeous, no doubt about that. But she\'s simply not a compelling screen presence. I\'ve always found myself wanting to like her more than I actually do, and this movie is no exception. She\'s simply not a very good actress, and she\'s not even a very good dancer. Good looking as she is, there\'s something vapid about her, and this movie suffers because of it.<br /><br />Grade: C-')

In [29]:
classifier(text)

[{'label': 'NEGATIVE', 'score': 0.9893576502799988}]